# Grounding Line Picker Demo

This notebook demonstrates the `frame.pick` xarray accessor for interactive grounding line picking on radar echograms.

**Features:**

- Interactive point picking with click-to-add
- Layer overlays (surface, bottom) with visibility toggles
- Snap-to-layer functionality for precise picking
- Undo/Clear controls
- CSV export for picked points

In [ ]:
import numpy as np
import holoviews as hv
import panel as pn
import xopr

# Import accessor to register it
import xopr_viewer  # noqa: F401

hv.extension("bokeh")
pn.extension()

## Connect to OPR

Establish an OPR session with a local cache directory for faster subsequent requests.

In [ ]:
# Establish an OPR session with caching
opr = xopr.OPRConnection(cache_dir="radar_cache")

## Load Radar Data

Query and load frames for a specific segment.

In [ ]:
# Select a segment
selected_collection = "2022_Antarctica_BaslerMKB"
selected_segment = "20230109_01"
print(f"Selected segment: {selected_segment}")

# Query frames
stac_items = opr.query_frames(
    collections=[selected_collection], segment_paths=[selected_segment]
)
print(f"Found {len(stac_items)} frames")
stac_items.head(3)

In [ ]:
# Load the radar data
frames = opr.load_frames(stac_items)
print(f"Loaded {len(frames)} frames")

## Merge Frames

Combine individual frames into a continuous flight line for easier visualization.

In [ ]:
# Merge frames into a single flight line
flight_line = xopr.merge_frames(frames)
flight_line

## Preprocess Data

Resample to uniform time spacing for faster plotting and convert to dB scale for visualization.

In [ ]:
# Resample to uniform spacing (faster plotting)
frame = flight_line.resample(slow_time="2s").mean()

# Convert to dB scale for visualization
frame["Data"] = 10 * np.log10(np.abs(frame["Data"]))
frame

## Load Layers

Get existing layer picks (surface, bottom) from OPR.

In [ ]:
# Load layers for the merged flight line
layers = opr.get_layers(frame)
print(f"Available layers: {list(layers.keys())}")

In [ ]:
# Inspect surface layer
if "standard:surface" in layers:
    layers["standard:surface"]

## Static Plot with Layers

Use `frame.pick.plot()` to create a static echogram with layer overlays.

In [ ]:
# Static plot with layer overlays
frame.pick.plot(layers=layers, width=900, height=400, log_scale=False)

## Interactive Picker Panel

Use `frame.pick.panel()` for the full interactive picking interface:

- **Layer checkboxes**: Toggle visibility of surface/bottom layers
- **Snap to layer**: When enabled, clicks snap to the nearest visible layer
- **Slope checkboxes**: Toggle a slope subplot below the echogram for any layer
- **Smoothing slider**: Control the rolling-mean window size for slope smoothing
- **Picks counter**: Shows current number of picked points
- **Undo/Clear**: Remove last point or clear all
- **Export**: Save picks to CSV

**Click on the echogram to add picks!**

## Layer Slope

Compute and visualize the slope (gradient) of layer profiles. Smoothing reduces noise before taking the derivative. This is useful for identifying regions where the bed or surface geometry changes rapidly.

In [ ]:
from xopr_viewer.picker import compute_layer_slope, _create_layer_curves

# Show the bottom layer profile alongside its slope
bottom = layers["standard:bottom"]
slope_raw = compute_layer_slope(bottom.twtt, smoothing_window=1)
slope_smooth = compute_layer_slope(bottom.twtt, smoothing_window=21)

bottom_curve = _create_layer_curves(layers, visible_layers=["standard:bottom"])

profile = bottom_curve["standard:bottom"].opts(
    width=900, height=200, title="Bottom Layer Profile"
)
raw_plot = hv.Curve(slope_raw, label="raw").opts(
    width=900,
    height=200,
    title="Bottom Slope (raw vs smoothed)",
    color="gray",
    alpha=0.5,
)
smooth_plot = hv.Curve(slope_smooth, label="smoothed").opts(
    width=900, height=200, color="yellow"
)

(profile + raw_plot * smooth_plot).cols(1)

In [ ]:
frame.pick.panel(layers=layers, width=900, height=400, log_scale=False)

## Using the Picker Programmatically

For more control, use `frame.pick.picker()` to get a `GroundingLinePicker` instance directly. This allows you to access picks as a DataFrame and export them.

In [ ]:
# Create a picker instance
picker = frame.pick.picker(layers=layers, log_scale=False)

# Display the interactive element
picker.panel(width=900, height=400)

## View and Export Picks

After clicking on the echogram above, access picks as a DataFrame or export to CSV.

In [ ]:
# View picks as DataFrame
print(f"Number of picks: {len(picker.points)}")
picker.df

## Save and Load Picks

Export picks to CSV and reload them in a future session.

In [ ]:
# Export picks to CSV
# picker.to_csv("picks.csv")

# Load picks from a previous session
# picker.from_csv("picks.csv")

## Picking Workflow

Typical workflow for labeling grounding points:

1. **Load radar data** and preprocess for visualization
2. **Load layers** (surface, bottom) for reference
3. **Open the picker** using `frame.pick.panel(layers=layers)`
4. **Enable snap** (optional): Check "Snap to layer" to snap clicks to visible layers
5. **Click on echogram** to add picks at grounding point locations
6. **Export to CSV** using the Export button or `picker.to_csv()`

The exported CSV contains:

- `id`: Unique identifier for each pick
- `slow_time`: Time coordinate (x-axis)
- `twtt_us`: Two-way travel time in microseconds (y-axis)